In [1]:
import numpy as np

In [ ]:
x = np.random.randint(-5, 10, (5, 3))
print(f'原始矩阵{x.shape}：')
print(x)
print('='*20)

X = x.T
print(f'原始矩阵的转置矩阵{X.shape}：')
print(X)
print('='*20)

mu = X.mean(axis=1)[:, None]
print(f'转置矩阵的按行平均值{mu.shape}：')
print(mu)
print('='*20)

A = X - mu
print(f'转置矩阵的按行进行均值零化{A.shape}：')
print(A)
print('='*20)

C = (A @ A.T) / x.shape[0]
print(f'协方差矩阵{C.shape}：')
print(C)
print('='*20)

D, V = np.linalg.eig(C)
idx = D.argsort()[::-1]
D = D[idx]
V = V[:, idx]
print(f'协方差矩阵的特征值：{D}')
print(f'协方差矩阵的特征向量{V.shape}：')
print(V)
print('='*20)

P = V.T
print(f'P is {P.shape}：')
print(P)
print('='*20)

print(f'协方差对角矩阵 = P @ C @ P.T')
print(P @ C @ P.T)
print('='*20)